In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_validate
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import scipy.stats
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
import statistics
%matplotlib inline
import sklearn
from collections import Counter
import itertools
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.utils import resample
from sklearn import linear_model
import math
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor

In [9]:
df = pd.read_csv("LIDC with IH(MSE).csv", index_col = "Unnamed: 0")
df.head()

,noduleID,Area,ConvexArea,Perimeter,ConvexPerimeter,EquivDiameter,MajorAxisLength,MinorAxisLength,Elongation,Compactness,...,Malignancy_2,Malignancy_3,Malignancy_4,Malignancy3_1,Malignancy3_2,Malignancy3_3,Malignancy3_4,Malignancy3_mode,Propagation,IH
0,1,1094,1286,168.852814,135.372918,37.321898,42.803687,35.834900,1.194469,2.073908,...,5,5,4,3,3,3,3,3,2,0.009204
1,3,931,1062,148.267027,119.799290,34.429435,39.635819,30.844618,1.285016,1.879012,...,5,3,4,3,3,2,3,3,2,0.011045
2,4,161,167,51.455844,49.201081,14.317527,18.125068,11.574663,1.565926,1.308681,...,4,3,2,3,3,2,1,3,3,0.001381
3,5,246,251,58.769553,57.721132,17.697936,18.324991,17.321312,1.057945,1.117274,...,5,3,2,3,3,2,1,3,1,0.126093
4,6,71,73,32.142136,31.494737,9.507892,10.528352,8.908660,1.181811,1.157927,...,1,1,1,1,1,1,1,1,3,0.002761


In [10]:
#df = df[df.IH != 0]
df.loc[df.IH == 0, 'IH'] = 0.0001
df.head()

,noduleID,Area,ConvexArea,Perimeter,ConvexPerimeter,EquivDiameter,MajorAxisLength,MinorAxisLength,Elongation,Compactness,...,Malignancy_2,Malignancy_3,Malignancy_4,Malignancy3_1,Malignancy3_2,Malignancy3_3,Malignancy3_4,Malignancy3_mode,Propagation,IH
0,1,1094,1286,168.852814,135.372918,37.321898,42.803687,35.834900,1.194469,2.073908,...,5,5,4,3,3,3,3,3,2,0.009204
1,3,931,1062,148.267027,119.799290,34.429435,39.635819,30.844618,1.285016,1.879012,...,5,3,4,3,3,2,3,3,2,0.011045
2,4,161,167,51.455844,49.201081,14.317527,18.125068,11.574663,1.565926,1.308681,...,4,3,2,3,3,2,1,3,3,0.001381
3,5,246,251,58.769553,57.721132,17.697936,18.324991,17.321312,1.057945,1.117274,...,5,3,2,3,3,2,1,3,1,0.126093
4,6,71,73,32.142136,31.494737,9.507892,10.528352,8.908660,1.181811,1.157927,...,1,1,1,1,1,1,1,1,3,0.002761


In [11]:
X = df.drop(["noduleID",'Malignancy_1', 'Malignancy_2', 'Malignancy_3', 'Malignancy_4',
    "Malignancy3_1", 'Malignancy3_2', "Malignancy3_3", "Malignancy3_4", "Malignancy3_mode", "Propagation", "IH"], axis=1)
X = X.as_matrix()
IH = df[["IH"]]
IH = IH.as_matrix()

In [12]:
def subsample(dataset, ratio=0.632):
    sample = list()
    n_sample = round(len(dataset) * ratio)
   # random.seed(10)
    sample_indices = np.random.choice(len(dataset), n_sample)
    for i in range(n_sample):
        index = sample_indices[i]
        sample.append(dataset[index])
    sample_asarray = np.asarray(sample)
    return sample_asarray

def hard_subsample(dataset, ih, hard_quotient, ratio=0.632):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    # Set weights to be a function of IH by adding a certain percentage of IH (as specified by hard_quotient) to weights
    w = np.ones(len(dataset)) + [hard_quotient*i for i in ih]   # This way the weights are between 1 and 1.2 if hard_quotient = 0.2 (default)
    weights = w / sum(w) # Now sum of all weights equal to one
    
  #  np.random.seed(10)
    sample_indices = np.random.choice(len(dataset), n_sample, p = weights, replace=True)
    for i in range(n_sample):
        index = sample_indices[i]
        sample.append(dataset[index])
    sample_asarray = np.asarray(sample)
    return sample_asarray

def easy_subsample(dataset, ih, hard_quotient, ratio=0.632):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    # Set weights to be a function of IH by adding a certain percentage of IH (as specified by hard_quotient) to weights
    easiness = [1 - i for i in ih]
    w = np.ones(len(dataset)) + [hard_quotient*i for i in easiness]   # This way the weights are between 1 and 1.2 if hard_quotient = 0.2 (default)
    weights = w / sum(w) # Now sum of all weights equal to one
    
  #  np.random.seed(10)
    sample_indices = np.random.choice(len(dataset), n_sample, p = weights, replace=True)
    for i in range(n_sample):
        index = sample_indices[i]
        sample.append(dataset[index])
    sample_asarray = np.asarray(sample)
    return sample_asarray

In [13]:
defnumbags = 10 # default number of bags
def create_regular_bags(trainset, nbags):
    bags = list()
    
    for i in range(nbags):
        bags.append(subsample(trainset))
    
    bags_asarray = np.asarray(bags)
    return bags_asarray

def create_mixed_bags(trainset, ih, nbags, mix_ratio, hard_quotient):
    neasy = round(nbags*mix_ratio[0]) 
    nnormal = round(nbags*mix_ratio[1]) 
    nhard = round(nbags*mix_ratio[2]) 
    
    bags = list()
    # Make easy bags
    for i in range(neasy):     # First nneasy easy bags    
        bags.append(easy_subsample(trainset, ih, hard_quotient))      
    # Make normal bags
    for i in range(neasy, neasy + nnormal):      # The next nnormal normal bags      
        bags.append(subsample(trainset))
    # Make hard bags
    for i in range(neasy + nnormal, neasy + nnormal + nhard):   # The next nhard hard bags
        bags.append(hard_subsample(trainset, ih, hard_quotient))
    
    bags_asarray = np.asarray(bags)
    return bags_asarray

def create_gradually_mixed_bags(trainset, ih, nbags, low_bag_hardness, high_bag_hardness):
    bag_hardness_values = np.linspace(low_bag_hardness, high_bag_hardness, nbags)   # Divide up the range to find hardness value for each bag
    
    bags = list()
    
    for i in range(nbags):   # nbags = len(hardness_values) 
        if bag_hardness_values[i] < 0:
            bags.append(easy_subsample(trainset, ih, 0 - bag_hardness_values[i]))  # say, bag hardness is -0.3 Thats the same as creating an easy bag with HC =  0.3
        elif bag_hardness_values[i] == 0:
            bags.append(subsample(trainset))
        else:   # if > 1
            bags.append(hard_subsample(trainset, ih, bag_hardness_values[i]))  # say, bag_hardness is 0.4. Thats the same as creating a hard bag with HC = 0.4
    
    bags_asarray = np.asarray(bags)
    return bags_asarray   

In [14]:
def make_models(bags, clf, X_test):
    nbags = bags.shape[0]
    nfeatures = bags.shape[2] - 2 # because the last two columns are y and ih
    
    predictions = list()  
    predictions_proba = list()

    for i in range(nbags):
        # Fit and predict (hard as well as soft predictions)
        clf.fit(bags[i][:,0:nfeatures], bags[i][:,nfeatures])
        predicted = clf.predict(X_test)
       # print(predicted.shape)
        predictions.append(predicted)
        pred_proba = clf.predict_proba(X_test)

        # Export tree as DOT file for visualization
    #    outfilename = "tree_bag"+ str(i) + ".dot" 
    #    export_graphviz(clf, out_file=outfilename)   # tree_bag1.dot, tree_bag2.dot etc.
        
        predictions_asarray = np.asarray(predictions)
    return predictions_asarray

def calculate_pred(predictions):
    ninst = predictions.shape[1] # No. of instances
    nbags = predictions.shape[0] # No. of bags
    
    # hard (binary) predictions
    final_pred = np.zeros((ninst, 1))
    for j in range(ninst):    # for each instance
        count = 0
        for i in range(nbags):    # find prediction in each bag and do majority vote
            if predictions[i, j] == 1:
                count = count + 1
        final_pred[j] = 1 if count > nbags/5 else 0   # Majority vote

    final_pred = []
    for prediction in predictions.T:
        tp = Counter(prediction)
        final_pred.append(tp.most_common()[0][0])

    final_pred = np.array(final_pred)
    
    return final_pred

In [15]:
# min_samples_split = 2  min_samples_leaf = 1
y_all = df[['Malignancy_1', 'Malignancy_2', 'Malignancy_3', 'Malignancy_4']]
y_all = y_all.as_matrix()
y_allt = np.transpose(y_all)
np.random.shuffle(y_allt)
y_all = np.transpose(y_allt)
y = y_all.T[0]

    
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

train_indexs = []
test_indexs = []
    

for train_index, test_index in skf.split(X, y):  
    train_indexs.append(train_index)
    test_indexs.append(test_index)

    
y_all_train = y_all[train_indexs[0]]
y_all_test = y_all[test_indexs[0]]
    
X_train = X[train_indexs[0]]
X_test = X[test_indexs[0]]

IH_train = IH[train_indexs[0]]
IH_test = IH[test_indexs[0]]
    
##First Iteration

y_train1 = np.zeros(y[train_indexs[0]].shape[0])
y_test1 = np.zeros(y[test_indexs[0]].shape[0])
    
#train
n = 0
for index in train_indexs[0]:
    y_train1[n] = y_all[index][0]
    n += 1 
     
#test
n = 0
for index in test_indexs[0]:
    y_test1[n] = y_all[index][0]
    n += 1
    
#Only for train
y_train1 = np.reshape(y_train1, (y_train1.shape[0], 1)).astype(int)
X_train_withlabels = np.concatenate((X_train, y_train1, IH_train), axis=1)

In [16]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

In [108]:
#have 20 versions of labels
def Mixed_Bagging_SIC(nbags, mix_ratio, hard_quotient):
    number  = 1

    train_acc1 = []
    test_acc1 = []
    noftree1 = []

    train_acc2 = []
    test_acc2 = []
    noftree2 = []

    train_acc3 = []
    test_acc3 = []
    noftree3 = []

    train_acc4 = []
    test_acc4 = []
    noftree4 = []

    train_acc1_classes = []
    train_acc2_classes = []
    train_acc3_classes = []
    train_acc4_classes = []

    train_sen1_classes = []
    train_sen2_classes = []
    train_sen3_classes = []
    train_sen4_classes = []

    train_spe1_classes = []
    train_spe2_classes = []
    train_spe3_classes = []
    train_spe4_classes = []

    test_acc1_classes = []
    test_acc2_classes = []
    test_acc3_classes = []
    test_acc4_classes = []

    test_sen1_classes = []
    test_sen2_classes = []
    test_sen3_classes = []
    test_sen4_classes = []

    test_spe1_classes = []
    test_spe2_classes = []
    test_spe3_classes = []
    test_spe4_classes = []


    cost1 = []
    cost2 = []
    cost3 = []
    cost4 = []
    
    trans_IH = np.log(IH)/np.log(10)
    linreg = LinearRegression()
    linreg.fit(X,trans_IH)

    while number <= 20 :

        y_all = df[['Malignancy_1', 'Malignancy_2', 'Malignancy_3', 'Malignancy_4']]
        y_all = y_all.as_matrix()
        y_allt = np.transpose(y_all)
        np.random.shuffle(y_allt)
        y_all = np.transpose(y_allt)
        y = y_all.T[0]

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

        train_indexs = []
        test_indexs = []


        for train_index, test_index in skf.split(X, y):  
            train_indexs.append(train_index)
            test_indexs.append(test_index)

        X_train = X[train_indexs[0]]
        X_test = X[test_indexs[0]]
        
        y_all_train = y_all[train_indexs[0]]
        y_all_test = y_all[test_indexs[0]]
        
        IH_train = IH[train_indexs[0]]
        
        ##First Iteration

        y_train1 = np.zeros(y[train_indexs[0]].shape[0])
        y_test1 = np.zeros(y[test_indexs[0]].shape[0])

        #train
        n = 0
        for index in train_indexs[0]:
            y_train1[n] = y_all[index][0]
            n += 1 

        #test
        n = 0
        for index in test_indexs[0]:
            y_test1[n] = y_all[index][0]
            n += 1
        
        #Predction  
        sm = SMOTE(random_state=2)
        X_train_balanced, y_train1_balanced = sm.fit_sample(X_train, y_train1)
        #predict synthetic IH
        p = linreg.predict(X_train_balanced[X_train.shape[0]:])
        p = np.exp(p*np.log(10))
        IH_train_balanced = np.concatenate((IH_train, p), axis=0)
        
        y_train1_balanced = np.reshape(y_train1_balanced, (y_train1_balanced.shape[0], 1)).astype(int)
        X_train_withlabels = np.concatenate((X_train_balanced, y_train1_balanced, IH_train_balanced), axis=1)

        mis_train_1 = []
        mis_test_1 = []

        mixed_bags = create_mixed_bags(X_train_withlabels, IH_train_balanced.flatten(),
                                       nbags, mix_ratio, hard_quotient)
        clf = DecisionTreeClassifier(max_depth= 5)
        train_pred = make_models(mixed_bags, clf, X_train)
        test_pred = make_models(mixed_bags, clf, X_test)
        bagging_train_pred= calculate_pred(train_pred)
        bagging_test_pred= calculate_pred(test_pred)
        
        train_acc1.append(accuracy_score(y_train1, bagging_train_pred))
        test_acc1.append(accuracy_score(y_test1, bagging_test_pred))

        #train
        cm = confusion_matrix(y_train1, bagging_train_pred, labels=[1, 2, 3, 4, 5])
        train_acc_class = []
        train_sen_class = []
        train_spe_class = []

        for i in range(5):  
            TP = cm[i][i]
            FN = cm[i].sum() - TP
            FP = cm.T[i].sum() - TP
            TN = cm.sum() - TP - FN - FP
            accuracy = (TP+TN)/cm.sum()
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)

            train_acc_class.append(accuracy)
            train_sen_class.append(sensitivity)
            train_spe_class.append(specificity)

        train_acc1_classes.append(train_acc_class)
        train_sen1_classes.append(train_sen_class)
        train_spe1_classes.append(train_spe_class)

        #test
        cm = confusion_matrix(y_test1, bagging_test_pred, labels=[1, 2, 3, 4, 5])
        test_acc_class = []
        test_sen_class = []
        test_spe_class = []

        for i in range(5):  
            TP = cm[i][i]
            FN = cm[i].sum() - TP
            FP = cm.T[i].sum() - TP
            TN = cm.sum() - TP - FN - FP
            accuracy = (TP+TN)/cm.sum()
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)

            test_acc_class.append(accuracy)
            test_sen_class.append(sensitivity)
            test_spe_class.append(specificity)

        test_acc1_classes.append(test_acc_class)
        test_sen1_classes.append(test_sen_class)
        test_spe1_classes.append(test_spe_class)  

        #calculate errors
        mis_train_error_1 = abs(y_train1 - bagging_train_pred)
        mis_test_error_1 = abs(y_test1 - bagging_test_pred)

        #track
        #train
        for i, j in zip(mis_train_error_1, train_indexs[0]):
            if i != 0:
                mis_train_1.append(j) 

        #test
        for i, j in zip(mis_test_error_1, test_indexs[0]):
            if i != 0:
                mis_test_1.append(j)

        cost1.append(len(mis_train_1)+len(mis_test_1))

        ##Second Iteration

        #Update labels

        y_train2 = np.zeros(y[train_indexs[0]].shape[0])
        y_test2 = np.zeros(y[test_indexs[0]].shape[0])

        #train
        n = 0
        for a, b in zip(mis_train_error_1, y_all_train):                                
            if a != 0:
                y_train2[n] = int((b[0] + b[1])/2)
            else:
                y_train2[n] = y_train1[n]
            n += 1   

        #test
        n = 0
        for a, b in zip(mis_test_error_1, y_all_test):                                
            if a != 0:
                y_test2[n] = int((b[0] + b[1])/2)
            else:
                y_test2[n] = y_test1[n]
            n += 1           


        #Predction
        #Balanced
        X_train_balanced, y_train2_balanced = sm.fit_sample(X_train, y_train2)
        #predict synthetic IH
        p = linreg.predict(X_train_balanced[X_train.shape[0]:])
        p = np.exp(p*np.log(10))
        IH_train_balanced = np.concatenate((IH_train, p), axis=0)
        
        y_train2_balanced = np.reshape(y_train2_balanced, (y_train2_balanced.shape[0], 1)).astype(int)
        X_train_withlabels = np.concatenate((X_train_balanced, y_train2_balanced, IH_train_balanced), axis=1)

        mis_train_2 = []
        mis_test_2 = []

        mixed_bags = create_mixed_bags(X_train_withlabels, IH_train_balanced.flatten(), 
                                       nbags, mix_ratio, hard_quotient)
        clf = DecisionTreeClassifier(max_depth= 5)
        train_pred = make_models(mixed_bags, clf, X_train)
        test_pred = make_models(mixed_bags, clf, X_test)
        bagging_train_pred= calculate_pred(train_pred)
        bagging_test_pred= calculate_pred(test_pred)
        
        train_acc2.append(accuracy_score(y_train2, bagging_train_pred))
        test_acc2.append(accuracy_score(y_test2, bagging_test_pred))

        #train
        cm = confusion_matrix(y_train2, bagging_train_pred, labels=[1, 2, 3, 4, 5])
        train_acc_class = []
        train_sen_class = []
        train_spe_class = []

        for i in range(5):  
            TP = cm[i][i]
            FN = cm[i].sum() - TP
            FP = cm.T[i].sum() - TP
            TN = cm.sum() - TP - FN - FP
            accuracy = (TP+TN)/cm.sum()
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)

            train_acc_class.append(accuracy)
            train_sen_class.append(sensitivity)
            train_spe_class.append(specificity)

        train_acc2_classes.append(train_acc_class)
        train_sen2_classes.append(train_sen_class)
        train_spe2_classes.append(train_spe_class)

        #test
        cm = confusion_matrix(y_test2, bagging_test_pred, labels=[1, 2, 3, 4, 5])
        test_acc_class = []
        test_sen_class = []
        test_spe_class = []

        for i in range(5):  
            TP = cm[i][i]
            FN = cm[i].sum() - TP
            FP = cm.T[i].sum() - TP
            TN = cm.sum() - TP - FN - FP
            accuracy = (TP+TN)/cm.sum()
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)

            test_acc_class.append(accuracy)
            test_sen_class.append(sensitivity)
            test_spe_class.append(specificity)

        test_acc2_classes.append(test_acc_class)
        test_sen2_classes.append(test_sen_class)
        test_spe2_classes.append(test_spe_class)  

        #calculate errors
        mis_train_error_2 = abs(y_train2 - bagging_train_pred)
        mis_test_error_2 = abs(y_test2 - bagging_test_pred)

        c = 0
        for i, j in zip(mis_train_error_2, train_indexs[0]):
            if j not in mis_train_1:
                mis_train_error_2[c] = 0
                mis_train_error_1[c] = i
            c += 1

        c = 0
        for i, j in zip(mis_test_error_2, test_indexs[0]):
            if j not in mis_test_1:
                mis_test_error_2[c] = 0
                mis_test_error_1[c] = i
            c += 1

        #track
        #train
        for i, j in zip(mis_train_error_2, train_indexs[0]):
            if i != 0:
                mis_train_2.append(j)

        #test
        for i, j in zip(mis_test_error_2, test_indexs[0]):
            if i != 0:
                mis_test_2.append(j)

        cost2.append(len(mis_train_2)+len(mis_test_2))

        ##Third Iteration

        #Update labels

        y_train3 = np.zeros(y[train_indexs[0]].shape[0])
        y_test3 = np.zeros(y[test_indexs[0]].shape[0])  

        #train
        n = 0
        for a, b, c in zip(mis_train_error_2, y_all_train, train_indexs[0]):                                
            if a != 0:
                #3 labels
                if c in mis_train_1 and c in mis_train_2:
                    if b[0] != b[1] and b[0] != b[2] and b[1] != b[2]:
                        y_train3[n] = int((b[0] + b[1] + b[2])/3)
                    else:
                        y_train3[n] = statistics.mode([int(b[0]),int(b[1]),int(b[2])])
                #2 labels
                else:
                    y_train3[n] = int((b[0] + b[1])/2)
            else:
                y_train3[n] = y_train2[n]
            n += 1  

        #test
        n = 0
        for a, b, c in zip(mis_test_error_2, y_all_test, test_indexs[0]):                                
            if a != 0:
                #3 labels
                if c in mis_test_1 and c in mis_test_2:
                    if b[0] != b[1] and b[0] != b[2] and b[1] != b[2]:
                        y_test3[n] = int((b[0] + b[1] + b[2])/3)
                    else:
                        y_test3[n] = statistics.mode([int(b[0]),int(b[1]),int(b[2])])
                #2 labels
                else:
                    y_test3[n] = int((b[0] + b[1])/2)
            else:
                y_test3[n] = y_test2[n]
            n += 1


        #Prediction
        #Balanced
        X_train_balanced, y_train3_balanced = sm.fit_sample(X_train, y_train3)
        #predict synthetic IH
        p = linreg.predict(X_train_balanced[X_train.shape[0]:])
        p = np.exp(p*np.log(10))
        IH_train_balanced = np.concatenate((IH_train, p), axis=0)
        
        y_train3_balanced = np.reshape(y_train3_balanced, (y_train3_balanced.shape[0], 1)).astype(int)
        X_train_withlabels = np.concatenate((X_train_balanced, y_train3_balanced, IH_train_balanced), axis=1)

        mis_train_3 = []
        mis_test_3 = []

        mixed_bags = create_mixed_bags(X_train_withlabels, IH_train_balanced.flatten(), 
                                       nbags, mix_ratio, hard_quotient)
        clf = DecisionTreeClassifier(max_depth= 5)
        train_pred = make_models(mixed_bags, clf, X_train)
        test_pred = make_models(mixed_bags, clf, X_test)
        bagging_train_pred= calculate_pred(train_pred)
        bagging_test_pred= calculate_pred(test_pred)
        
        train_acc3.append(accuracy_score(y_train3, bagging_train_pred))
        test_acc3.append(accuracy_score(y_test3, bagging_test_pred))

        #train
        cm = confusion_matrix(y_train3, bagging_train_pred, labels=[1, 2, 3, 4, 5])
        train_acc_class = []
        train_sen_class = []
        train_spe_class = []

        for i in range(5):  
            TP = cm[i][i]
            FN = cm[i].sum() - TP
            FP = cm.T[i].sum() - TP
            TN = cm.sum() - TP - FN - FP
            accuracy = (TP+TN)/cm.sum()
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)

            train_acc_class.append(accuracy)
            train_sen_class.append(sensitivity)
            train_spe_class.append(specificity)

        train_acc3_classes.append(train_acc_class)
        train_sen3_classes.append(train_sen_class)
        train_spe3_classes.append(train_spe_class)

        #test
        cm = confusion_matrix(y_test3, bagging_test_pred, labels=[1, 2, 3, 4, 5])
        test_acc_class = []
        test_sen_class = []
        test_spe_class = []

        for i in range(5):  
            TP = cm[i][i]
            FN = cm[i].sum() - TP
            FP = cm.T[i].sum() - TP
            TN = cm.sum() - TP - FN - FP
            accuracy = (TP+TN)/cm.sum()
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)

            test_acc_class.append(accuracy)
            test_sen_class.append(sensitivity)
            test_spe_class.append(specificity)

        test_acc3_classes.append(test_acc_class)
        test_sen3_classes.append(test_sen_class)
        test_spe3_classes.append(test_spe_class)  

        #calculate errors
        mis_train_error_3 = abs(y_train3 - bagging_train_pred)
        mis_test_error_3 = abs(y_test3 - bagging_test_pred)

        c = 0
        for i, j in zip(mis_train_error_3, train_indexs[0]):
            if j not in mis_train_1 and j not in mis_train_2:
                mis_train_error_3[c] = 0
                mis_train_error_1[c] = i
            elif j not in mis_train_1 and j in mis_train_2:
                mis_train_error_3[c] = 0
                mis_train_error_2[c] = i
            elif j in mis_train_1 and j not in mis_train_2:
                mis_train_error_3[c] = 0
                mis_train_error_2[c] = i
            c += 1

        c = 0
        for i, j in zip(mis_test_error_3, test_indexs[0]):
            if j not in mis_test_1 and j not in mis_test_2:
                mis_test_error_3[c] = 0
                mis_test_error_1[c] = i
            elif j not in mis_test_1 and j in mis_test_2:
                mis_test_error_3[c] = 0
                mis_test_error_2[c] = i
            elif j in mis_test_1 and j not in mis_test_2:
                mis_test_error_3[c] = 0
                mis_test_error_2[c] = i
            c += 1

        #track
        #train
        for i, j in zip(mis_train_error_3, train_indexs[0]):
            if i != 0:
                mis_train_3.append(j)

        #test
        for i, j in zip(mis_test_error_3, test_indexs[0]):
            if i != 0:
                mis_test_3.append(j)

        cost3.append(len(mis_train_3)+len(mis_test_3))

        ##Fourth Iteration

        #Update labels

        y_train4 = np.zeros(y[train_indexs[0]].shape[0])
        y_test4 = np.zeros(y[test_indexs[0]].shape[0])  

        #train
        n = 0
        for a, b, c in zip(mis_train_error_3, y_all_train, train_indexs[0]):                                
            if a != 0:
                #4 labels
                if c in mis_train_1 and c in mis_train_2 and c in mis_train_3:
                    if b[3] == b[0] and b[1] == b[2]:    
                        y_train4[n] = int((b[3] + b[0] + b[1] + b[2])/4)
                    elif b[3] == b[1] and b[0] == b[2]:    
                        y_train4[n] = int((b[3] + b[0] + b[1] + b[2])/4)
                    elif b[3] == b[2] and b[0] == b[1]:    
                        y_train4[n] = int((b[3] + b[0] + b[1] + b[2])/4)
                    elif b[3] != b[0] and b[3] != b[1] and b[3] != b[2] and b[0] != b[1] and b[0] != b[2] and b[1] != b[2]:
                        y_train4[n] = int((b[3] + b[0] + b[1] + b[2])/4)
                    else:
                        y_train4[n] = statistics.mode([int(b[0]),int(b[3]),int(b[1]), int(b[2])])

                # 3 labels
                elif c in mis_train_1 and c in mis_train_3:
                    if b[0] != b[1] and b[0] != b[2] and b[1] != b[2]:
                        y_train4[n] = int((b[0] + b[1] + b[2])/3)
                    else:
                        y_train4[n] = statistics.mode([int(b[0]),int(b[1]),int(b[2])])


                #3 labels
                elif c in mis_train_2 and c in mis_train_3:
                    if b[0] != b[1] and b[0] != b[2] and b[1] != b[2]:
                        y_train4[n] = int((b[0] + b[1] + b[2])/3)
                    else:
                        y_train4[n] = statistics.mode([int(b[0]),int(b[1]),int(b[2])])


                #2 labels
                else:
                    y_train4[n] = int((b[0] + b[1])/2)
            else:
                y_train4[n] = y_train3[n]

            n += 1  


        #test
        n = 0
        for a, b, c in zip(mis_test_error_3, y_all_test, test_indexs[0]):                                
            if a != 0:
                #4 labels
                if c in mis_test_1 and c in mis_test_2 and c in mis_test_3:
                    if b[3] == b[0] and b[1] == b[2]:    
                        y_test4[n] = int((b[3] + b[0] + b[1] + b[2])/4)
                    elif b[3] == b[1] and b[0] == b[2]:    
                        y_test4[n] = int((b[3] + b[0] + b[1] + b[2])/4)
                    elif b[3] == b[2] and b[0] == b[1]:    
                        y_test4[n] = int((b[3] + b[0] + b[1] + b[2])/4)
                    elif b[3] != b[0] and b[3] != b[1] and b[3] != b[2] and b[0] != b[1] and b[0] != b[2] and b[1] != b[2]:
                        y_test4[n] = int((b[3] + b[0] + b[1] + b[2])/4)
                    else:
                        y_test4[n] = statistics.mode([int(b[0]),int(b[3]),int(b[1]), int(b[2])])

                # 3 labels
                elif c in mis_test_1 and c in mis_test_3:
                    if b[0] != b[1] and b[0] != b[2] and b[1] != b[2]:
                        y_test4[n] = int((b[0] + b[1] + b[2])/3)
                    else:
                        y_test4[n] = statistics.mode([int(b[0]),int(b[1]),int(b[2])])


                #3 labels
                elif c in mis_test_2 and c in mis_test_3:
                    if b[0] != b[1] and b[0] != b[2] and b[1] != b[2]:
                        y_test4[n] = int((b[0] + b[1] + b[2])/3)
                    else:
                        y_test4[n] = statistics.mode([int(b[0]),int(b[1]),int(b[2])])

                #2 labels
                else:
                    y_test4[n] = int((b[0] + b[1])/2)
            else:
                y_test4[n] = y_test3[n]

            n += 1  

        #Prediction
        #Balanced
        X_train_balanced, y_train4_balanced = sm.fit_sample(X_train, y_train4)
        #predict synthetic IH
        p = linreg.predict(X_train_balanced[X_train.shape[0]:])
        p = np.exp(p*np.log(10))
        IH_train_balanced = np.concatenate((IH_train, p), axis=0)
        
        y_train4_balanced = np.reshape(y_train4_balanced, (y_train4_balanced.shape[0], 1)).astype(int)
        X_train_withlabels = np.concatenate((X_train_balanced, y_train4_balanced, IH_train_balanced), axis=1)

        mis_train_4 = []
        mis_test_4 = []

        mixed_bags = create_mixed_bags(X_train_withlabels, IH_train_balanced.flatten(), 
                                       nbags, mix_ratio, hard_quotient)
        clf = DecisionTreeClassifier(max_depth= 5)
        train_pred = make_models(mixed_bags, clf, X_train)
        test_pred = make_models(mixed_bags, clf, X_test)
        bagging_train_pred= calculate_pred(train_pred)
        bagging_test_pred= calculate_pred(test_pred)
        
        train_acc4.append(accuracy_score(y_train4, bagging_train_pred))
        test_acc4.append(accuracy_score(y_test4, bagging_test_pred))

        #train
        cm = confusion_matrix(y_train4, bagging_train_pred, labels=[1, 2, 3, 4, 5])
        train_acc_class = []
        train_sen_class = []
        train_spe_class = []

        for i in range(5):  
            TP = cm[i][i]
            FN = cm[i].sum() - TP
            FP = cm.T[i].sum() - TP
            TN = cm.sum() - TP - FN - FP
            accuracy = (TP+TN)/cm.sum()
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)

            train_acc_class.append(accuracy)
            train_sen_class.append(sensitivity)
            train_spe_class.append(specificity)

        train_acc4_classes.append(train_acc_class)
        train_sen4_classes.append(train_sen_class)
        train_spe4_classes.append(train_spe_class)

        #test
        cm = confusion_matrix(y_test4, bagging_test_pred, labels=[1, 2, 3, 4, 5])
        test_acc_class = []
        test_sen_class = []
        test_spe_class = []

        for i in range(5):  
            TP = cm[i][i]
            FN = cm[i].sum() - TP
            FP = cm.T[i].sum() - TP
            TN = cm.sum() - TP - FN - FP
            accuracy = (TP+TN)/cm.sum()
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)

            test_acc_class.append(accuracy)
            test_sen_class.append(sensitivity)
            test_spe_class.append(specificity)

        test_acc4_classes.append(test_acc_class)
        test_sen4_classes.append(test_sen_class)
        test_spe4_classes.append(test_spe_class)  

        #calculate errors
        mis_train_error_4 = abs(y_train4 - bagging_train_pred)
        mis_test_error_4 = abs(y_test4 - bagging_test_pred)

        c = 0
        for i, j in zip(mis_train_error_4, train_indexs[0]):
            if j not in mis_train_1 and j not in mis_train_2 and j not in mis_train_3:
                mis_train_error_4[c] = 0
                mis_train_error_1[c] = i
            elif j in mis_train_1 and j not in mis_train_2 and j not in mis_train_3:
                mis_train_error_4[c] = 0
                mis_train_error_2[c] = i
            elif j not in mis_train_1 and j in mis_train_2 and j not in mis_train_3:
                mis_train_error_4[c] = 0
                mis_train_error_2[c] = i
            elif j not in mis_train_1 and j not in mis_train_2 and j in mis_train_3:
                mis_train_error_4[c] = 0
                mis_train_error_2[c] = i
            elif j in mis_train_1 and j in mis_train_2 and j not in mis_train_3:
                mis_train_error_4[c] = 0
                mis_train_error_2[c] = i
            elif j not in mis_train_1 and j in mis_train_2 and j in mis_train_3:
                mis_train_error_4[c] = 0
                mis_train_error_2[c] = i
            elif j in mis_train_1 and j not in mis_train_2 and j in mis_train_3:
                mis_train_error_4[c] = 0
                mis_train_error_2[c] = i
            c += 1

        c = 0
        for i, j in zip(mis_test_error_4, test_indexs[0]):
            if j not in mis_test_1 and j not in mis_test_2 and j not in mis_test_3:
                mis_test_error_4[c] = 0
                mis_test_error_1[c] = i
            elif j in mis_test_1 and j not in mis_test_2 and j not in mis_test_3:
                mis_test_error_4[c] = 0
                mis_test_error_2[c] = i
            elif j not in mis_test_1 and j in mis_test_2 and j not in mis_test_3:
                mis_test_error_4[c] = 0
                mis_test_error_2[c] = i
            elif j not in mis_test_1 and j not in mis_test_2 and j in mis_test_3:
                mis_test_error_4[c] = 0
                mis_test_error_2[c] = i
            elif j in mis_test_1 and j in mis_test_2 and j not in mis_test_3:
                mis_test_error_4[c] = 0
                mis_test_error_2[c] = i
            elif j not in mis_test_1 and j in mis_test_2 and j in mis_test_3:
                mis_test_error_4[c] = 0
                mis_test_error_2[c] = i
            elif j in mis_test_1 and j not in mis_test_2 and j in mis_test_3:
                mis_test_error_4[c] = 0
                mis_test_error_2[c] = i
            c += 1

        #track
        #train
        for i, j in zip(mis_train_error_4, train_indexs[0]):
            if i != 0:
                mis_train_4.append(j)

        #test
        for i, j in zip(mis_test_error_4, test_indexs[0]):
            if i != 0:
                mis_test_4.append(j)

        cost4.append(len(mis_train_4)+len(mis_test_4))  

        print(number)
        number += 1

    ave_acc1 = round(sum(test_acc1)/len(test_acc1),4)*100
    ave_acc2 = round(sum(test_acc2)/len(test_acc2),4)*100
    ave_acc3 = round(sum(test_acc3)/len(test_acc3),4)*100
    ave_acc4 = round(sum(test_acc4)/len(test_acc4),4)*100

    print("\n","testing accuracy", ave_acc1, ave_acc2, ave_acc3, ave_acc4)
    print("\n","all testing accuracies", mean_confidence_interval(test_acc1), mean_confidence_interval(test_acc2), mean_confidence_interval(test_acc3), mean_confidence_interval(test_acc4))

    #Iteration 1 Sensitivity
    class1 = []
    class2 = []
    class3 = []
    class4 = []
    class5 = []
    for I in train_sen1_classes:
        class1.append(I[0])
        class2.append(I[1])
        class3.append(I[2])
        class4.append(I[3])
        class5.append(I[4])

    print("\n", "Iteration1 train sensitivity", "class1", round(sum(class1)/len(class1),4)*100, "class2", round(sum(class2)/len(class2),4)*100,
          "class3", round(sum(class3)/len(class3),4)*100, "class4", round(sum(class4)/len(class4),4)*100, "class5", round(sum(class5)/len(class5),4)*100)

    #Iteration 2 Sensitivity
    class1 = []
    class2 = []
    class3 = []
    class4 = []
    class5 = []
    for I in train_sen2_classes:
        class1.append(I[0])
        class2.append(I[1])
        class3.append(I[2])
        class4.append(I[3])
        class5.append(I[4])

    print("\n", "Iteration2 train sensitivity", "class1", round(sum(class1)/len(class1),4)*100, "class2", round(sum(class2)/len(class2),4)*100,
          "class3", round(sum(class3)/len(class3),4)*100, "class4", round(sum(class4)/len(class4),4)*100, "class5", round(sum(class5)/len(class5),4)*100)

    #Iteration 3 Sensitivity
    class1 = []
    class2 = []
    class3 = []
    class4 = []
    class5 = []
    for I in train_sen3_classes:
        class1.append(I[0])
        class2.append(I[1])
        class3.append(I[2])
        class4.append(I[3])
        class5.append(I[4])

    print("\n", "Iteration3 train sensitivity", "class1", round(sum(class1)/len(class1),4)*100, "class2", round(sum(class2)/len(class2),4)*100,
          "class3", round(sum(class3)/len(class3),4)*100, "class4", round(sum(class4)/len(class4),4)*100, "class5", round(sum(class5)/len(class5),4)*100)

    #Iteration 4 Sensitivity
    class1 = []
    class2 = []
    class3 = []
    class4 = []
    class5 = []
    for I in train_sen4_classes:
        class1.append(I[0])
        class2.append(I[1])
        class3.append(I[2])
        class4.append(I[3])
        class5.append(I[4])

    print("\n", "Iteration4 train sensitivity", "class1", round(sum(class1)/len(class1),4)*100, "class2", round(sum(class2)/len(class2),4)*100,
          "class3", round(sum(class3)/len(class3),4)*100, "class4", round(sum(class4)/len(class4),4)*100, "class5", round(sum(class5)/len(class5),4)*100)

    #Iteration 1 Sensitivity
    class1 = []
    class2 = []
    class3 = []
    class4 = []
    class5 = []
    for I in test_sen1_classes:
        class1.append(I[0])
        class2.append(I[1])
        class3.append(I[2])
        class4.append(I[3])
        class5.append(I[4])

    print("\n", "Iteration1 test sensitivity", "class1", round(sum(class1)/len(class1),4)*100, "class2", round(sum(class2)/len(class2),4)*100,
          "class3", round(sum(class3)/len(class3),4)*100, "class4", round(sum(class4)/len(class4),4)*100, "class5", round(sum(class5)/len(class5),4)*100)

    #Iteration 2 Sensitivity
    class1 = []
    class2 = []
    class3 = []
    class4 = []
    class5 = []
    for I in test_sen2_classes:
        class1.append(I[0])
        class2.append(I[1])
        class3.append(I[2])
        class4.append(I[3])
        class5.append(I[4])

    print("\n", "Iteration2 test sensitivity", "class1", round(sum(class1)/len(class1),4)*100, "class2", round(sum(class2)/len(class2),4)*100,
          "class3", round(sum(class3)/len(class3),4)*100, "class4", round(sum(class4)/len(class4),4)*100, "class5", round(sum(class5)/len(class5),4)*100)

    #Iteration 3 Sensitivity
    class1 = []
    class2 = []
    class3 = []
    class4 = []
    class5 = []
    for I in test_sen3_classes:
        class1.append(I[0])
        class2.append(I[1])
        class3.append(I[2])
        class4.append(I[3])
        class5.append(I[4])

    print("\n", "Iteration3 test sensitivity", "class1", round(sum(class1)/len(class1),4)*100, "class2", round(sum(class2)/len(class2),4)*100,
          "class3", round(sum(class3)/len(class3),4)*100, "class4", round(sum(class4)/len(class4),4)*100, "class5", round(sum(class5)/len(class5),4)*100)

    #Iteration 4 Sensitivity
    class1 = []
    class2 = []
    class3 = []
    class4 = []
    class5 = []
    for I in test_sen4_classes:
        class1.append(I[0])
        class2.append(I[1])
        class3.append(I[2])
        class4.append(I[3])
        class5.append(I[4])

    print("\n", "Iteration4 test sensitivity", "class1", round(sum(class1)/len(class1),4)*100, mean_confidence_interval(class1),
          "class2", round(sum(class2)/len(class2),4)*100, mean_confidence_interval(class2),
          "class3", round(sum(class3)/len(class3),4)*100, mean_confidence_interval(class3),
          "class4", round(sum(class4)/len(class4),4)*100, mean_confidence_interval(class4),
          "class5", round(sum(class5)/len(class5),4)*100, mean_confidence_interval(class5))
    print("\n", "all class 1", class1)
    print("\n", "all class 2", class2)
    print("\n", "all class 3", class3)
    print("\n", "all class 4", class4)
    print("\n", "all class 5", class5)
    
    
    
    
    print("\n", "cost", int(815 + sum(cost1)/len(cost1) + sum(cost2)/len(cost2) + sum(cost3)/len(cost3)))